In [ ]:
import whisper
import torch
import pandas as pd
import os

# Check for cuda/cpu
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device}')

# Load the model: large-v3-turbo or large-v3
model = whisper.load_model("large-v3-turbo", device=device)

CUDA (GPU) is available. Initializing model on GPU.


Transcribe the given song, create list of "curses" words to search for

In [ ]:
# I put a few tracks in here
song = 'juvenile'
audio_file = f"music/{song}.mp3"

# Format file path
notebook_dir = os.getcwd()
full_audio_path = os.path.join(notebook_dir, audio_file)

# word_timestamps=True for timestamp info
# Getting Triton kernel issues when I run this ?
result = model.transcribe(full_audio_path, word_timestamps=True)

c:\Users\dacla\anaconda3\Lib\site-packages\whisper\timing.py:42: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower median kernel implementation...
  warnings.warn(
c:\Users\dacla\anaconda3\Lib\site-packages\whisper\timing.py:146: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower DTW implementation...
  warnings.warn(


In [ ]:
# Edit for things to look for
curses = {'fuck', 'motherfucker', 'shit', 'bitch', 'nigga'}
to_add = []

# Collect words
for segment in result["segments"]:
    for word_info in segment['words']:
        for curse in curses:
            if curse in word_info['word'].strip().lower():
                to_add.append([word_info['word'].strip().lower(), [float(word_info['start']), float(word_info['end'])], word_info['probability']])
        
# This can probably be done better
# There needs to be some amount of text editing done to the automatic transcriptions:
# - Remove punctuation, make lowercase, separate root instances of common words 


# Create Dataframe
columns = ['Word', 'Time', 'Prob']
df = pd.DataFrame(to_add, columns=columns)

,Word,Time,Prob
0,fuck,"[6.02, 6.48]",0.960418
1,motherfuckers,"[74.3, 75.24]",0.996993
2,motherfuckers,"[74.3, 75.24]",0.996993


In [41]:
# Inspect dataframe
df.head()

,Word,Time,Prob
0,fuck,"[6.02, 6.48]",0.960418
1,motherfuckers,"[74.3, 75.24]",0.996993


In [22]:
## Print full transcript
full_text = result["text"].strip()
print(full_text)

I can truly say I don't give a fuck about your money Because it means so much to you and all of your money Why does it mean so much to you? You cite the land of greed And I'm talking about a world of need Money has nothing to do with the value of life But that's just common sense You call me up and you talk about money You call me up and you talk about money You call me up and you talk about money I don't want to hear anymore about money Look what people do, trying to get their money Defy the fantasy based of the book of the movie Fortified cares and modified principles Convenience is the name of the game Self-indulgence is the rule Selfish motherfuckers There are no winners We all lose We all lose They fall not set You call me up and you talk about money You call me up and you talk about money You call me up and you talk about money The way you pay for my money You call me up and you talk about money You call me up and you talk about money родs


In [ ]:
# Pydub does the audio effects
from pydub import AudioSegment

def reverse_audio_segment(input_audio_path, output_audio_path, times):
    """
    Reverses a specific segment of an audio track and saves the modified track.

    Args:
        input_audio_path (str): Path to the original audio file.
        output_audio_path (str): Path to save the modified audio file.
        times (list: int): Segments to reverse (in ms)
    """
    # Load the audio file
    audio = AudioSegment.from_file(input_audio_path)
    for (start_ms, end_ms) in times:
        
    # Ensure start and end times are within the audio duration
        if start_ms < 0 or end_ms > len(audio) or start_ms >= end_ms:
            print("Invalid start/end times. Please check the duration of the audio.")
            print(f"Audio duration: {len(audio)} ms")
            return

        # Select times to reverse
        before_segment = audio[:start_ms]

        target_segment = audio[start_ms:end_ms]
        reversed_segment = target_segment.reverse()

        after_segment = audio[end_ms:]

        #Concatenate the segments: before + reversed + after
        audio = before_segment + reversed_segment + after_segment
        #print(f'Reversed: {[start_ms, end_ms]}')

    # Export the modified audio
    audio.export(output_audio_path, format="mp3") 
    return


In [ ]:
# Path formatting
notebook_dir = os.getcwd()

input_file = f"music/{song}.mp3"
output_file = f"music/{song}-edit.mp3"

input_path = os.path.join(notebook_dir, input_file)
output_path = os.path.join(notebook_dir, output_file)

## Maybe a bit inefficient
times = []
for row in df.itertuples():
    word = row[1]
    start, end = row[2]
    times.append((int(start*1000), int(end*1000)))

# Run the reversing script
reverse_audio_segment(input_path, output_path, times)

Reversed: [6020, 6480]
Reversed: [74300, 75240]


Ideas: 
- demucs to split audio tracks

